In [3]:
from pymongo import MongoClient
import json
import pandas as pd
import numpy as np
from bson import json_util


client = MongoClient('mongodb://localhost:27017/')
db = client.companies

In [4]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

In [5]:
# Primero quiero obtener un listado de todas las empresas que han quebrado para evitar esas localizaciones
# de esta manera pretendo establecer criterios a la inversa.

companies_broken = db.companies.find({'$and':[
                                      {'deadpooled_year':{'$not':{'$eq':None}}}, 
                                      {'offices.latitude':{'$exists':True}},
                                      {'offices.longitude':{'$exists':True}}, 
                                      {'offices.latitude':{'$ne':None}},
                                      {'offices.longitude':{'$ne':None}}]})

In [6]:
companies_broken = pd.DataFrame(list(companies_broken))
companies_broken.head(2)

,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297d8b,None,[],Zoho ManageEngine,,,enterprise,[],2007-05-25 19:24:22,http://www.crunchbase.com/company/adventnet,...,925-924-9500,[],"[{'provider': {'name': 'A Small Orange', 'perm...","[{'person': {'last_name': 'Vembu', 'permalink'...","[{'attribution': None, 'available_sizes': [[[1...",,$0,manageengine,Wed Oct 31 18:26:09 UTC 2012,[]
1,52cdef7c4bab8bd675297d8a,"{'acquiring_company': {'name': 'Viggle', 'perm...",[],,http://digitalquarters.net/feed/,http://digitalquarters.net/,web,"[{'competitor': {'name': 'Wikia', 'permalink':...",2007-05-25 06:51:27,http://www.crunchbase.com/company/wetpaint,...,206.859.6300,"[{'name': 'Wikison Wetpaint', 'permalink': 'we...",[],"[{'person': {'last_name': 'Howell', 'permalink...","[{'attribution': None, 'available_sizes': [[[1...","wiki, seattle, elowitz, media-industry, media-...",$39.8M,BachelrWetpaint,Sun Dec 08 07:15:44 UTC 2013,[]


In [7]:
companies_broken.set_index('_id', inplace=True)
companies_broken.head(2)


,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,deadpooled_day,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
_id,,,,,,,,,,,,,,,,,,,,,
52cdef7c4bab8bd675297d8b,None,[],Zoho ManageEngine,,,enterprise,[],2007-05-25 19:24:22,http://www.crunchbase.com/company/adventnet,NaN,...,925-924-9500,[],"[{'provider': {'name': 'A Small Orange', 'perm...","[{'person': {'last_name': 'Vembu', 'permalink'...","[{'attribution': None, 'available_sizes': [[[1...",,$0,manageengine,Wed Oct 31 18:26:09 UTC 2012,[]
52cdef7c4bab8bd675297d8a,"{'acquiring_company': {'name': 'Viggle', 'perm...",[],,http://digitalquarters.net/feed/,http://digitalquarters.net/,web,"[{'competitor': {'name': 'Wikia', 'permalink':...",2007-05-25 06:51:27,http://www.crunchbase.com/company/wetpaint,NaN,...,206.859.6300,"[{'name': 'Wikison Wetpaint', 'permalink': 'we...",[],"[{'person': {'last_name': 'Howell', 'permalink...","[{'attribution': None, 'available_sizes': [[[1...","wiki, seattle, elowitz, media-industry, media-...",$39.8M,BachelrWetpaint,Sun Dec 08 07:15:44 UTC 2013,[]


In [8]:
# Para poder ver el número de cada columna que quiero eliminar, he recurrido a un loop for con un enumerate
# de esta forma obtengo el número de cada columna para así eliminar las que no me interesan en el siguiente paso
'''
for x,y in enumerate (companies_broken.columns):
    print (x, " - ", y)
'''
#Como ocupa mucho, una vez cumplida su función lo dejo como anotación para que no ocupe mucho

'\nfor x,y in enumerate (companies_broken.columns):\n    print (x, " - ", y)\n'

In [9]:
#Eliminamos las columnas que no nos son de utilidad

companies_broken.drop(companies_broken.columns[[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 14, 15, 16, 17, 20, 21, 22, 24, 28, 29, 30, 31, 32, 33, 34, 35, 38, 39, 40
]], axis=1, inplace=True)

companies_broken.head(2)

,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,name,number_of_employees,offices,tag_list,total_money_raised
_id,,,,,,,,,,,
52cdef7c4bab8bd675297d8b,enterprise,2,Server Management Software,1996.0,[],NaN,AdventNet,600.0,"[{'city': 'Pleasanton', 'latitude': 37.692934,...",,$0
52cdef7c4bab8bd675297d8a,web,1,Technology Platform Company,2005.0,"[{'funded_day': 1, 'round_code': 'a', 'investm...",NaN,Wetpaint,47.0,"[{'city': 'Seattle', 'latitude': 47.603122, 'l...","wiki, seattle, elowitz, media-industry, media-...",$39.8M


In [10]:
# Ahora vamos a establecer la geolocalizacion, para luego ponerla en el geoindex

def getFirst(data):
    data = data['offices']

    principal = None
    if data[0]['latitude'] and data[0]['longitude']:
        principal = {
            "type":"Point",
            "coordinates":[data[0]['longitude'], data[0]['latitude']]
        }

    return {
        "totalOffices": len(data),
        "lat": data[0]['latitude'],
        "lng": data[0]['longitude'],
        "oficina_principal": principal
    }


comp_br_off = companies_broken[["offices"]].apply(getFirst, result_type="expand", axis=1)

In [11]:
companies_broken = pd.concat([companies_broken, comp_br_off], axis=1)
companies_broken.head(2)

,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,name,number_of_employees,offices,tag_list,total_money_raised,lat,lng,oficina_principal,totalOffices
_id,,,,,,,,,,,,,,,
52cdef7c4bab8bd675297d8b,enterprise,2,Server Management Software,1996.0,[],NaN,AdventNet,600.0,"[{'city': 'Pleasanton', 'latitude': 37.692934,...",,$0,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,...",1
52cdef7c4bab8bd675297d8a,web,1,Technology Platform Company,2005.0,"[{'funded_day': 1, 'round_code': 'a', 'investm...",NaN,Wetpaint,47.0,"[{'city': 'Seattle', 'latitude': 47.603122, 'l...","wiki, seattle, elowitz, media-industry, media-...",$39.8M,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,...",2


In [12]:
# Ahora guardamos el dataframe en un json

companies_broken.to_json('./companies_broken.json', orient='records', default_handler=str)

In [13]:
# Ahora vamos a hacer un listado con las empresas que no cumplen con nuestros requisitos para 
# tenerlas ubicadas y así poder establecer descartes en base a la cercanía a estas empresas.

companies_dislike = db.companies.find({'$and':
                                      [{'deadpooled_year':{'$eq':None}},{'founded_year':{'$lt':2008}},
                                       {"offices.latitude":{'$exists':True}},{"offices.longitude":
                                       {'$exists':True}},{"offices.latitude":{'$ne':None}},
                                       {"offices.longitude":{'$ne':None}}]})


In [14]:
companies_dislike = pd.DataFrame(list(companies_dislike))
companies_dislike.head(2)

,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297d8d,"{'acquiring_company': {'name': 'betaworks', 'p...","[{'acquired_day': 15, 'company': {'name': 'Wef...",,http://blog.digg.com/?feed=rss2,http://blog.digg.com/,news,"[{'competitor': {'name': 'Reddit', 'permalink'...",Fri May 25 20:03:23 UTC 2007,http://www.crunchbase.com/company/digg,...,(415) 436-9638,"[{'name': 'Digg', 'permalink': 'digg'}]","[{'provider': {'name': 'K/F Communications', '...","[{'person': {'last_name': 'McLaughlin', 'perma...","[{'attribution': None, 'available_sizes': [[[1...","community, social, news, bookmark, digg, techn...",$45M,digg,Tue Nov 05 21:35:47 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl..."
1,52cdef7c4bab8bd675297d90,"{'acquiring_company': {'name': 'Google', 'perm...",[],None,,,web,"[{'competitor': {'name': 'RescueTime', 'permal...",Fri Jun 08 12:19:51 UTC 2007,http://www.crunchbase.com/company/postini,...,888.584.3150,"[{'name': 'Postini Solutions', 'permalink': 'p...",[],"[{'person': {'last_name': 'Petry', 'permalink'...",[],,$0,None,Sat Aug 13 18:02:34 UTC 2011,[]


In [15]:
companies_dislike.set_index('_id', inplace=True)
companies_dislike.head(2)

,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,deadpooled_day,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
_id,,,,,,,,,,,,,,,,,,,,,
52cdef7c4bab8bd675297d8d,"{'acquiring_company': {'name': 'betaworks', 'p...","[{'acquired_day': 15, 'company': {'name': 'Wef...",,http://blog.digg.com/?feed=rss2,http://blog.digg.com/,news,"[{'competitor': {'name': 'Reddit', 'permalink'...",Fri May 25 20:03:23 UTC 2007,http://www.crunchbase.com/company/digg,None,...,(415) 436-9638,"[{'name': 'Digg', 'permalink': 'digg'}]","[{'provider': {'name': 'K/F Communications', '...","[{'person': {'last_name': 'McLaughlin', 'perma...","[{'attribution': None, 'available_sizes': [[[1...","community, social, news, bookmark, digg, techn...",$45M,digg,Tue Nov 05 21:35:47 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl..."
52cdef7c4bab8bd675297d90,"{'acquiring_company': {'name': 'Google', 'perm...",[],None,,,web,"[{'competitor': {'name': 'RescueTime', 'permal...",Fri Jun 08 12:19:51 UTC 2007,http://www.crunchbase.com/company/postini,None,...,888.584.3150,"[{'name': 'Postini Solutions', 'permalink': 'p...",[],"[{'person': {'last_name': 'Petry', 'permalink'...",[],,$0,None,Sat Aug 13 18:02:34 UTC 2011,[]


In [16]:
#Eliminamos las columnas que no nos son de utilidad

companies_dislike.drop(companies_dislike.columns[[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 14, 15, 16, 17, 20, 21, 22, 24, 28, 29, 30, 31, 32, 33, 34, 35, 38, 39, 40
]], axis=1, inplace=True)

companies_dislike.head(2)

,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,name,number_of_employees,offices,tag_list,total_money_raised
_id,,,,,,,,,,,
52cdef7c4bab8bd675297d8d,news,None,user driven social content website,2004,"[{'funded_day': 1, 'round_code': 'b', 'investm...",None,Digg,60.0,"[{'city': 'San Francisco', 'latitude': 37.7647...","community, social, news, bookmark, digg, techn...",$45M
52cdef7c4bab8bd675297d90,web,None,None,1999,[],None,Postini,NaN,"[{'city': 'San Carlos', 'latitude': 37.506885,...",,$0


In [17]:
# Ahora vamos a establecer la geolocalizacion, para luego ponerla en el geoindex

comp_dislike_off = companies_dislike[["offices"]].apply(getFirst, result_type="expand", axis=1)

In [18]:
companies_dislike = pd.concat([companies_dislike, comp_dislike_off], axis=1)
companies_dislike.head(2)

,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,name,number_of_employees,offices,tag_list,total_money_raised,lat,lng,oficina_principal,totalOffices
_id,,,,,,,,,,,,,,,
52cdef7c4bab8bd675297d8d,news,None,user driven social content website,2004,"[{'funded_day': 1, 'round_code': 'b', 'investm...",None,Digg,60.0,"[{'city': 'San Francisco', 'latitude': 37.7647...","community, social, news, bookmark, digg, techn...",$45M,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,...",1
52cdef7c4bab8bd675297d90,web,None,None,1999,[],None,Postini,NaN,"[{'city': 'San Carlos', 'latitude': 37.506885,...",,$0,37.506885,-122.247573,"{'type': 'Point', 'coordinates': [-122.247573,...",1


In [19]:
# Ahora guardamos el dataframe en un json

companies_dislike.to_json('./companies_dislike.json', orient='records', default_handler=str)

In [20]:
#Aquí hacemos un filtro de las empresas que pueden encajar con nuestros requisitos


filtered_companies = db.companies.find({'$and': 
                                        [{'deadpooled_year':{'$eq':None}}, {'founded_year':{'$gte':2008}},
                                         {"offices.latitude":{'$exists':True}},{"offices.longitude":{'$exists':True}},
                                         {"offices.latitude":{'$ne':None}},{"offices.longitude":{'$ne':None}}]})


In [21]:
filtered_companies = pd.DataFrame(list(filtered_companies))
filtered_companies.head(2)


,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297e30,None,[],None,,http://goingon.com/blog,software,"[{'competitor': {'name': 'Ning by Glam Media',...",Fri Jul 20 04:55:26 UTC 2007,http://www.crunchbase.com/company/goingon,...,415-200-4606,"[{'name': 'GoingOn', 'permalink': 'goingon'}, ...",[],"[{'person': {'last_name': 'Corshen', 'permalin...",[],"socialnetworking, enterprise, social-media, so...",$8.5M,GoingOn_Network,Thu Apr 04 09:03:17 UTC 2013,[]
1,52cdef7c4bab8bd675297ea7,None,[],,,,web,"[{'competitor': {'name': 'Club Penguin', 'perm...",Tue Aug 07 05:47:56 UTC 2007,http://www.crunchbase.com/company/mokitown,...,,"[{'name': 'Mokitown', 'permalink': 'mokitown'}]",[],[],"[{'attribution': None, 'available_sizes': [[[1...","mokitown, virtualworld, crunchbase",$0,,Tue Mar 19 01:07:12 UTC 2013,[]


In [22]:
filtered_companies.set_index('_id', inplace=True)
filtered_companies.head(2)

,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,deadpooled_day,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
_id,,,,,,,,,,,,,,,,,,,,,
52cdef7c4bab8bd675297e30,None,[],None,,http://goingon.com/blog,software,"[{'competitor': {'name': 'Ning by Glam Media',...",Fri Jul 20 04:55:26 UTC 2007,http://www.crunchbase.com/company/goingon,None,...,415-200-4606,"[{'name': 'GoingOn', 'permalink': 'goingon'}, ...",[],"[{'person': {'last_name': 'Corshen', 'permalin...",[],"socialnetworking, enterprise, social-media, so...",$8.5M,GoingOn_Network,Thu Apr 04 09:03:17 UTC 2013,[]
52cdef7c4bab8bd675297ea7,None,[],,,,web,"[{'competitor': {'name': 'Club Penguin', 'perm...",Tue Aug 07 05:47:56 UTC 2007,http://www.crunchbase.com/company/mokitown,None,...,,"[{'name': 'Mokitown', 'permalink': 'mokitown'}]",[],[],"[{'attribution': None, 'available_sizes': [[[1...","mokitown, virtualworld, crunchbase",$0,,Tue Mar 19 01:07:12 UTC 2013,[]


In [23]:
#Eliminamos las columnas que no nos son de utilidad

filtered_companies.drop(filtered_companies.columns[[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 14, 15, 16, 17, 20, 21, 22, 24, 28, 29, 30, 31, 32, 33, 34, 35, 38, 39, 40
]], axis=1, inplace=True)

filtered_companies.head(2)

,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,name,number_of_employees,offices,tag_list,total_money_raised
_id,,,,,,,,,,,
52cdef7c4bab8bd675297e30,software,None,Private Academic Networks,2008,"[{'funded_day': 1, 'round_code': 'seed', 'inve...",None,GoingOn,40.0,"[{'city': 'San Francisco', 'latitude': 37.7822...","socialnetworking, enterprise, social-media, so...",$8.5M
52cdef7c4bab8bd675297ea7,web,None,,2011,[],None,Mokitown,NaN,"[{'city': None, 'latitude': 37.09024, 'longitu...","mokitown, virtualworld, crunchbase",$0


In [22]:
# Ahora vamos a establecer la geolocalizacion, para luego ponerla en el geoindex

fil_comp_off = filtered_companies[["offices"]].apply(getFirst, result_type="expand", axis=1)

In [23]:
filtered_companies = pd.concat([filtered_companies, fil_comp_off], axis=1)
filtered_companies.head(2)

,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,name,number_of_employees,offices,tag_list,total_money_raised,lat,lng,oficina_principal,totalOffices
_id,,,,,,,,,,,,,,,
52cdef7c4bab8bd675297e30,software,None,Private Academic Networks,2008,"[{'source_url': '', 'funded_day': 1, 'investme...",None,GoingOn,40.0,"[{'longitude': -122.392142, 'description': 'Go...","socialnetworking, enterprise, social-media, so...",$8.5M,37.782263,-122.392142,"{'type': 'Point', 'coordinates': [-122.392142,...",1
52cdef7c4bab8bd675297ea7,web,None,,2011,[],None,Mokitown,NaN,"[{'longitude': -95.712891, 'description': None...","mokitown, virtualworld, crunchbase",$0,37.090240,-95.712891,"{'type': 'Point', 'coordinates': [-95.712891, ...",1


In [24]:
filtered_companies.to_json('./filtered_companies.json', orient='records', default_handler=str)